In [1]:
from rdflib import Graph
g = Graph()
g.parse("C:/Users/laure/OneDrive/Documents/GitHub/KG_project/nutrition_structure_short.ttl", format="turtle")
#print(f"Graph has {len(g)} triples.")
# 3514824 triples

<Graph identifier=Nc7d4ba7412084f1b8fb66c7494f29a91 (<class 'rdflib.graph.Graph'>)>

# SPARKSQL QUERIES

In [2]:
# SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?food ?protein ?proteinAmount ?weight ?weightAmount
WHERE {
    ?food ex:hasNutrient ?protein .
    FILTER(STRSTARTS(STR(?protein), CONCAT(STR(ex:), "Protein_")))
    ?protein ex:hasWeight ?proteinAmount .

    ?food ex:hasNutrient ?weight .
    FILTER(STRSTARTS(STR(?weight), CONCAT(STR(ex:), "Weight_")))
    ?weight ex:hasWeight ?weightAmount .

    FILTER( (?proteinAmount / ?weightAmount) * 100 > 10)
}

"""

# Execute query
results = g.query(query)

# Print results
for row in results:
    # Ensure the values are floats for the calculation
    protein_amount = float(row.proteinAmount)
    weight_amount = float(row.weightAmount)
    normalized_protein = (protein_amount / weight_amount) * 100
    print(f"Food: {row.food}, Protein Amount: {normalized_protein}")


Food: http://example.org/Alaskan_King_Crab, Protein Amount: 19.294117647058822
Food: http://example.org/Beef_chuck_arm_pot_roast_separable_lean_only_trimmed_to_1_8__fat_choice_cooked_braised, Protein Amount: 34.691629955947135
Food: http://example.org/Beef_rib_eye_roast_bone-in_lip-on_separable_lean_only_trimmed_to_1_8__fat_select_cooked_roasted, Protein Amount: 27.029659141212925
Food: http://example.org/Dried_Eggs, Protein Amount: 48.0
Food: http://example.org/Duck_domesticated_meat_only_raw, Protein Amount: 18.283828382838283
Food: http://example.org/Egg_Omelet, Protein Amount: 10.491803278688524
Food: http://example.org/Fish_ling_raw, Protein Amount: 19.01554404145078
Food: http://example.org/Fish_salmon_chinook_smoked__lox__regular, Protein Amount: 18.235294117647058
Food: http://example.org/Fontina_Cheese, Protein Amount: 25.55555555555556
Food: http://example.org/Hard_Goat_Cheese, Protein Amount: 30.5
Food: http://example.org/Liver_cheese_pork, Protein Amount: 15.263157894736842

In [3]:
# SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?food ?proteinAmount ?caloriesAmount ?weightAmount
WHERE {
    ?food ex:hasNutrient ?protein .
    FILTER(STRSTARTS(STR(?protein), CONCAT(STR(ex:), "Protein_")))
    ?protein ex:hasWeight ?proteinAmount .

    ?food ex:hasNutrient ?weight .
    FILTER(STRSTARTS(STR(?weight), CONCAT(STR(ex:), "Weight_")))
    ?weight ex:hasWeight ?weightAmount .

    OPTIONAL {
        ?food ex:hasNutrient ?calorie .
        FILTER(STRSTARTS(STR(?calorie), CONCAT(STR(ex:), "Calories_")))
        ?calorie ex:hasWeight ?caloriesAmount .
    }

    FILTER ((?proteinAmount / ?weightAmount) * 100 > 10 && bound(?caloriesAmount) && (?caloriesAmount / ?weightAmount) * 100 < 300)
}

"""

# Execute query
results = g.query(query)

# Print results
for row in results:
    # Ensure the values are floats for the calculation
    protein_amount = float(row.proteinAmount)
    weight_amount = float(row.weightAmount)
    calories_amount = float(row.caloriesAmount)

    # Calculate the normalized values
    normalized_protein = (protein_amount / weight_amount) * 100
    normalized_calories = (calories_amount / weight_amount) * 100

    # Print the values along with their normalized values
    print(f"Food: {row.food}, Protein: {protein_amount}, Normalized Protein: {normalized_protein}, Calories: {calories_amount}, Normalized Calories: {normalized_calories}")


Food: http://example.org/Alaskan_King_Crab, Protein: 16.4, Normalized Protein: 19.294117647058822, Calories: 82.0, Normalized Calories: 96.47058823529412
Food: http://example.org/Beef_chuck_arm_pot_roast_separable_lean_only_trimmed_to_1_8__fat_choice_cooked_braised, Protein: 157.5, Normalized Protein: 34.691629955947135, Calories: 1016.0, Normalized Calories: 223.78854625550662
Food: http://example.org/Beef_rib_eye_roast_bone-in_lip-on_separable_lean_only_trimmed_to_1_8__fat_select_cooked_roasted, Protein: 610.6, Normalized Protein: 27.029659141212925, Calories: 4947.0, Normalized Calories: 218.9907038512616
Food: http://example.org/Duck_domesticated_meat_only_raw, Protein: 55.4, Normalized Protein: 18.283828382838283, Calories: 409.0, Normalized Calories: 134.98349834983497
Food: http://example.org/Egg_Omelet, Protein: 6.4, Normalized Protein: 10.491803278688524, Calories: 94.0, Normalized Calories: 154.0983606557377
Food: http://example.org/Fish_ling_raw, Protein: 36.7, Normalized Pr

In [4]:
# SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?food ?proteinAmount ?caloriesAmount
WHERE {
    ?food ex:hasNutrient ?protein .
    FILTER(STRSTARTS(STR(?protein), CONCAT(STR(ex:), "Protein_")))
    ?protein ex:hasWeight ?proteinAmount .

    OPTIONAL {
        ?food ex:hasNutrient ?calorie .
        FILTER(STRSTARTS(STR(?calorie), CONCAT(STR(ex:), "Calories_")))
        ?calorie ex:hasWeight ?caloriesAmount .
    }

    FILTER (
        ?proteinAmount > 0 && bound(?caloriesAmount) && ?caloriesAmount > 0 &&
        (?caloriesAmount / ?proteinAmount) < 10
    )
}

"""

# Execute query
results = g.query(query)

# Print results
for row in results:
    print(row.caloriesAmount)
    print(f"Food: {row.food}, Protein: {row.proteinAmount}, Calories: {row.caloriesAmount}")

82.0
Food: http://example.org/Alaskan_King_Crab, Protein: 16.4, Calories: 82.0
1016.0
Food: http://example.org/Beef_chuck_arm_pot_roast_separable_lean_only_trimmed_to_1_8__fat_choice_cooked_braised, Protein: 157.5, Calories: 1016.0
4947.0
Food: http://example.org/Beef_rib_eye_roast_bone-in_lip-on_separable_lean_only_trimmed_to_1_8__fat_select_cooked_roasted, Protein: 610.6, Calories: 4947.0
409.0
Food: http://example.org/Duck_domesticated_meat_only_raw, Protein: 55.4, Calories: 409.0
168.0
Food: http://example.org/Fish_ling_raw, Protein: 36.7, Calories: 168.0
99.0
Food: http://example.org/Fish_salmon_chinook_smoked__lox__regular, Protein: 15.5, Calories: 99.0
400.0
Food: http://example.org/Milk_dry_nonfat_calcium_reduced, Protein: 40.1, Calories: 400.0
185.0
Food: http://example.org/Pork_ground__minced__96__lean___4__fat_cooked_pan-broiled, Protein: 31.7, Calories: 185.0
477.0
Food: http://example.org/Tofu_dried-frozen__koyadofu_, Protein: 52.5, Calories: 477.0


## An average human of 70kg should have a daily intake of essential amino acids of : Histidine: 980 mg Isoleucine: 1330 mg Leucine: 2940 mg Lysine: 2660 mg Methionine: 1330 mg Phenylalanine: 2310 mg Threonine: 1400 mg Tryptophan: 350 mg Valine: 1680 mg
https://my.clevelandclinic.org/health/articles/22243-amino-acids


In [5]:

# Define the optimized SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?food ?weightAmount ?proteinAmount

       (SUM(?histidine) AS ?HistidineAmount)
       (SUM(?isoleucine) AS ?IsoleucineAmount)
       (SUM(?leucine) AS ?LeucineAmount)
       (SUM(?lysine) AS ?LysineAmount)
       (SUM(?methionine) AS ?MethionineAmount)
       (SUM(?phenylalanine) AS ?PhenylalanineAmount)
       (SUM(?threonine) AS ?ThreonineAmount)
       (SUM(?tryptophan) AS ?TryptophanAmount)
       (SUM(?valine) AS ?ValineAmount)
WHERE {
    ?food ex:hasNutrient ?nutrient .
    ?nutrient ex:hasWeight ?dv .

    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Histidine_")), ?dv, 0) AS ?histidine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Isoleucine_")), ?dv, 0) AS ?isoleucine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Leucine_")), ?dv, 0) AS ?leucine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Lysine_")), ?dv, 0) AS ?lysine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Methionine_")), ?dv, 0) AS ?methionine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Phenylalanine_")), ?dv, 0) AS ?phenylalanine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Threonine_")), ?dv, 0) AS ?threonine)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Tryptophan_")), ?dv, 0) AS ?tryptophan)
    BIND(IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), "Valine_")), ?dv, 0) AS ?valine)

    ?food ex:hasNutrient ?weight .
    FILTER(STRSTARTS(STR(?weight), CONCAT(STR(ex:), "Weight_")))
    ?weight ex:hasWeight ?weightAmount .

    ?food ex:hasNutrient ?protein .
    FILTER(STRSTARTS(STR(?protein), CONCAT(STR(ex:), "Protein_")))
    ?protein ex:hasWeight ?proteinAmount .
}
GROUP BY ?food
HAVING (
    SUM(?histidine) > 980 && SUM(?isoleucine) > 1330 && SUM(?leucine) > 2940 &&
    SUM(?lysine) > 2660 && SUM(?methionine) > 1330 && SUM(?phenylalanine) > 2310 &&
    SUM(?threonine) > 1400 && SUM(?tryptophan) > 350 && SUM(?valine) > 1680
)

"""

# Execute query
results = g.query(query)

# Print results
print("Foods with a complete essential amino acid profile:")
count = 0
for row in results:
    count += 1

    # Print the results with the normalized values
    print(f"""
    {count}. {row.food}
    Normalized Protein : {row.proteinAmount}
    Normalized Histidine: {row.HistidineAmount}
    Normalized Isoleucine: {row.IsoleucineAmount}
    Normalized Leucine: {row.LeucineAmount}
    Normalized Lysine: {row.LysineAmount}
    Normalized Methionine: {row.MethionineAmount}
    Normalized Phenylalanine: {row.PhenylalanineAmount}
    Normalized Threonine: {row.ThreonineAmount}
    Normalized Tryptophan: {row.TryptophanAmount}
    Normalized Valine: {row.ValineAmount}
    """)

Foods with a complete essential amino acid profile:

    1. http://example.org/Beef_chuck_arm_pot_roast_separable_lean_only_trimmed_to_1_8__fat_choice_cooked_braised
    Normalized Protein : 157.5
    Normalized Histidine: 5026.0
    Normalized Isoleucine: 7162.0
    Normalized Leucine: 12528.0
    Normalized Lysine: 13309.0
    Normalized Methionine: 4101.0
    Normalized Phenylalanine: 6219.0
    Normalized Threonine: 6291.0
    Normalized Tryptophan: 1034.0
    Normalized Valine: 7811.0
    

    2. http://example.org/Beef_rib_eye_roast_bone-in_lip-on_separable_lean_only_trimmed_to_1_8__fat_select_cooked_roasted
    Normalized Protein : 610.6
    Normalized Histidine: 27560.0
    Normalized Isoleucine: 33682.0
    Normalized Leucine: 62778.0
    Normalized Lysine: 70436.0
    Normalized Methionine: 19292.0
    Normalized Phenylalanine: 28780.0
    Normalized Threonine: 33998.0
    Normalized Tryptophan: 7952.0
    Normalized Valine: 35534.0
    

    3. http://example.org/Duck_domes

## FOOD combination that have a complete essential amino acid profile

In [6]:

# Define the optimized SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?food1 ?food2 ?totalProtein
       (SUM(?histidine1 + ?histidine2) AS ?HistidineAmount)
       (SUM(?isoleucine1 + ?isoleucine2) AS ?IsoleucineAmount)
       (SUM(?leucine1 + ?leucine2) AS ?LeucineAmount)
       (SUM(?lysine1 + ?lysine2) AS ?LysineAmount)
       (SUM(?methionine1 + ?methionine2) AS ?MethionineAmount)
       (SUM(?phenylalanine1 + ?phenylalanine2) AS ?PhenylalanineAmount)
       (SUM(?threonine1 + ?threonine2) AS ?ThreonineAmount)
       (SUM(?tryptophan1 + ?tryptophan2) AS ?TryptophanAmount)
       (SUM(?valine1 + ?valine2) AS ?ValineAmount)
WHERE {
    ?food1 ex:hasNutrient ?nutrient1 .
    ?nutrient1 ex:hasWeight ?dv1 .
    ?food2 ex:hasNutrient ?nutrient2 .
    ?nutrient2 ex:hasWeight ?dv2 .
    FILTER(?food1 != ?food2)  # Ensure different food items

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Histidine_")), ?dv1, 0) AS ?histidine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Histidine_")), ?dv2, 0) AS ?histidine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Isoleucine_")), ?dv1, 0) AS ?isoleucine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Isoleucine_")), ?dv2, 0) AS ?isoleucine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Leucine_")), ?dv1, 0) AS ?leucine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Leucine_")), ?dv2, 0) AS ?leucine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Lysine_")), ?dv1, 0) AS ?lysine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Lysine_")), ?dv2, 0) AS ?lysine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Methionine_")), ?dv1, 0) AS ?methionine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Methionine_")), ?dv2, 0) AS ?methionine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Phenylalanine_")), ?dv1, 0) AS ?phenylalanine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Phenylalanine_")), ?dv2, 0) AS ?phenylalanine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Threonine_")), ?dv1, 0) AS ?threonine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Threonine_")), ?dv2, 0) AS ?threonine2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Tryptophan_")), ?dv1, 0) AS ?tryptophan1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Tryptophan_")), ?dv2, 0) AS ?tryptophan2)

    BIND(IF(STRSTARTS(STR(?nutrient1), CONCAT(STR(ex:), "Valine_")), ?dv1, 0) AS ?valine1)
    BIND(IF(STRSTARTS(STR(?nutrient2), CONCAT(STR(ex:), "Valine_")), ?dv2, 0) AS ?valine2)

    ?food1 ex:hasNutrient ?protein1 .
    FILTER(STRSTARTS(STR(?protein1), CONCAT(STR(ex:), "Protein_")))
    ?protein1 ex:hasWeight ?proteinAmount1 .

    ?food2 ex:hasNutrient ?protein2 .
    FILTER(STRSTARTS(STR(?protein2), CONCAT(STR(ex:), "Protein_")))
    ?protein2 ex:hasWeight ?proteinAmount2 .

    BIND(?proteinAmount1 + ?proteinAmount2 AS ?totalProtein)
}
GROUP BY ?food1 ?food2 ?totalProtein
HAVING (
    SUM(?histidine1 + ?histidine2) > 980 &&
    SUM(?isoleucine1 + ?isoleucine2) > 1330 &&
    SUM(?leucine1 + ?leucine2) > 2940 &&
    SUM(?lysine1 + ?lysine2) > 2660 &&
    SUM(?methionine1 + ?methionine2) > 1330 &&
    SUM(?phenylalanine1 + ?phenylalanine2) > 2310 &&
    SUM(?threonine1 + ?threonine2) > 1400 &&
    SUM(?tryptophan1 + ?tryptophan2) > 350 &&
    SUM(?valine1 + ?valine2) > 1680
)

"""

# Execute query
results = g.query(query)

# Print results
print("Foods with a complete essential amino acid profile:")
count = 0
for row in results:
    count += 1

    # Print the results with the normalized values
    print(f"""
    {count}. {row.food1} + {row.food2}
    Normalized Protein : {row.proteinAmount}
    Normalized Histidine: {row.HistidineAmount}
    Normalized Isoleucine: {row.IsoleucineAmount}
    Normalized Leucine: {row.LeucineAmount}
    Normalized Lysine: {row.LysineAmount}
    Normalized Methionine: {row.MethionineAmount}
    Normalized Phenylalanine: {row.PhenylalanineAmount}
    Normalized Threonine: {row.ThreonineAmount}
    Normalized Tryptophan: {row.TryptophanAmount}
    Normalized Valine: {row.ValineAmount}
    """)

Foods with a complete essential amino acid profile:


KeyboardInterrupt: 

## Deficient in each Essential Amino Acids

In [7]:
# Define the SPARQL query
query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?aminoAcid (COUNT(?food) AS ?count)
WHERE {
    ?food ex:hasNutrient ?nutrient .
    ?nutrient ex:hasWeight ?dv .

    VALUES (?aminoAcid ?threshold) {
        ("Histidine" 980)
        ("Isoleucine" 1330)
        ("Leucine" 2940)
        ("Lysine" 2660)
        ("Methionine" 1330)
        ("Phenylalanine" 2310)
        ("Threonine" 1400)
        ("Tryptophan" 350)
        ("Valine" 1680)
    }

    BIND(
        IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), ?aminoAcid, "_")), ?dv, 0)
        AS ?amount
    )

    FILTER(SUM(?amount) < ?threshold)
}
GROUP BY ?aminoAcid
ORDER BY ?aminoAcid
"""

# Execute query
results = g.query(query)

# Print results
print("Amino Acid,Count")
for row in results:
    print(f"{row.aminoAcid},{row.count}")

Exception: What do I do with this CompValue? Aggregate_Sum_Aggregate_Sum_{'distinct': [], 'vars': rdflib.term.Variable('amount'), '_vars': {rdflib.term.Variable('amount')}}

In [8]:
# Query for Best Source (highest amount for each amino acid)
best_source_query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?aminoAcid ?bestFood (MAX(?amount) AS ?maxAmount)
WHERE {
    ?food ex:hasNutrient ?nutrient .
    ?nutrient ex:hasWeight ?dv .

    VALUES ?aminoAcid {
        "Protein" "Histidine" "Isoleucine" "Leucine" "Lysine" "Methionine" "Phenylalanine" "Threonine" "Tryptophan" "Valine"
    }

    BIND(
        IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), ?aminoAcid, "_")), ?dv, 0)
        AS ?amount
    )

    ?food ex:hasName ?bestFood .
}
GROUP BY ?aminoAcid ?bestFood
ORDER BY DESC(?maxAmount)  # Ordering by maxAmount so the highest value appears first
LIMIT 1
"""


# Query for Worst Source (lowest amount for each amino acid)
worst_source_query = """
PREFIX ex: <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?aminoAcid ?worstFood (MIN(?amount) AS ?minAmount)
WHERE {
    ?food ex:hasNutrient ?nutrient .
    ?nutrient ex:hasWeight ?dv .

    VALUES ?aminoAcid {
        "Protein" "Histidine" "Isoleucine" "Leucine" "Lysine" "Methionine" "Phenylalanine" "Threonine" "Tryptophan" "Valine"
    }

    BIND(
        IF(STRSTARTS(STR(?nutrient), CONCAT(STR(ex:), ?aminoAcid, "_")), ?dv, 0)
        AS ?amount
    )

    ?food ex:hasName ?worstFood .
}
GROUP BY ?aminoAcid ?worstFood
ORDER BY ASC(?minAmount)  # Ordering by minAmount so the lowest value appears first
LIMIT 1
"""

# Execute the Best Source query (highest amounts for each amino acid)
best_results = g.query(best_source_query)

# Execute the Worst Source query (lowest amounts for each amino acid)
worst_results = g.query(worst_source_query)

# Print the results for Best Source (Highest Amount)
print("Best Source for Amino Acids (Highest Amount):")
for row in best_results:
    print(f"{row.aminoAcid} {row.bestFood} {row.maxAmount}")

# Print the results for Worst Source (Lowest Amount)
print("\nWorst Source for Amino Acids (Lowest Amount):")
for row in worst_results:
    print(f"{row.aminoAcid} {row.worstFood} {row.minAmount}")

Best Source for Amino Acids (Highest Amount):
Lysine Beef_rib_eye_roast_bone-in_lip-on_separable_lean_only_trimmed_to_1_8__fat_select_cooked_roasted 70436.0

Worst Source for Amino Acids (Lowest Amount):
Protein Acorn_stew__Apache_ 0
